---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order according to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [4]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
type(df)

pandas.core.series.Series

In [17]:
from datetime import date;
import sys

abbv_months = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10,'Nov': 11, 'Dec': 12}
long_months = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6, 'July': 7, 'August': 8, 'September': 9, 'October': 10,'November': 11, 'December': 12}

def date_sorter():
    all_formatted_dates = []
    
    # datetime.date(year, month, day)
    my_strings = df.str.extractall(r'((\d{1,2})[/-](\d{1,2})[/-](\d{4}))')
    index = 0
    
    for string in my_strings[0]:
        if (str(string) != 'nan'):
            #print (string[:string.find('/')])
            #print (string[string.find('/')+1:string.rfind('/')])
            if (('/' in string) and (int(string[string.find('/')+1:string.rfind('/')]) < 32)):
                formatted = date(int(string[string.rfind('/')+1:]), int(string[:string.find('/')]), int(string[string.find('/')+1:string.rfind('/')]))
            
            elif ('-' in string) and (int(string[string.find('-')+1:string.rfind('-')]) < 32):
                formatted = date(int(string[string.rfind('-')+1:]), int(string[:string.find('-')]), int(string[string.find('-')+1:string.rfind('-')]))
            
            else:
                print ('Something is wrong')
                formatted = None
            
            if (formatted != None) and not(formatted in all_formatted_dates):
                all_formatted_dates.append(formatted)
                del formatted
            
        index += 1 
        
    my_strings = df.str.extractall(r'((\d{1,2})[/-](\d{1,2})[/-](\d{2}))')
    index = 0 
    
    for string in my_strings[0]:
        if (str(string) != 'nan'):
            if (('/' in string) and (int(string[string.find('/')+1:string.rfind('/')]) < 32)):
                formatted = date(int('19'+string[string.rfind('/')+1:]), int(string[:string.find('/')]), int(string[string.find('/')+1:string.rfind('/')]))
            
            elif ('-' in string) and (int(string[string.find('-')+1:string.rfind('-')]) < 32):
                formatted = date(int('19'+string[string.rfind('-')+1:]), int(string[:string.find('-')]), int(string[string.find('-')+1:string.rfind('-')]))
            
            else:
                print ('Something is wrong')
                formatted = None
            
            if (formatted != None) and not(formatted in all_formatted_dates):
                all_formatted_dates.append(formatted)
                del formatted
        index += 1 
 
    index = 0
    my_strings = df.str.extract(r'((Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec).? \d{1,2},? (\d{4}))')
    for string in my_strings[0]:
        if (str(string) != 'nan'):
            
            if (string.find('.') == -1):
                start = string.find(' ')
            else:
                start = min(string.find(' '), string.find('.'))
            
            year = int(string[end+1:])
            # (year, month, day)
            if (',' in str(string)):
                end = string.rfind(',') 
            else:
                end = string.rfind(' ')
            
            formatted = dates(year, abbv_months[string[:start]], int(string[start+1:end]))
            
        index += 1 
        
    my_strings = df.str.extract(r'((Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec).? ?\d{1,2}(th|rd|nd|st)?,? ?(\d{2}))')
        
    
    sys.exit(all_formatted_dates)
        
    index = 0
    my_strings = df.str.extract(r'((Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)-\d{1,2}(th|rd|nd|st)?-\d{2,4})')
    for string in my_strings[0]:
        if ((str(string)  != 'nan') and not(string in all_dates)):
            all_dates.append((string, index))
        index += 1 
        
        
    index = 0
    my_strings = df.str.extract(r'((January|February|March|April|May|June|July|August|September|October|November|December)-\d{1,2}(th|rd|nd|st)?-(\d{2,4}))')
    for string in my_strings[0]:
        if ((str(string)  != 'nan') and not(string in all_dates)):
            all_dates.append((string, index))
        index += 1 
        
        
    index = 0
    my_strings = df.str.extract(r'((January|February|March|April|May|June|July|August|September|October|November|December).? ?\d{1,2}(th|rd|nd|st)?,? ?(\d{2,4}))')
    for string in my_strings[0]:
        if ((str(string)  != 'nan') and not(string in all_dates)):
            all_dates.append((string, index))
        index += 1 
        
    index = 0
    my_strings = df.str.extract(r'((\d{1,2})[/ ](\d{4}|\d{2}))')
    for string in my_strings[0]:
        if ((str(string)  != 'nan') and not(string in all_dates)):
            all_dates.append((string, index))
        index += 1 
        
    index = 0        
    my_strings = df.str.extract(r'(\d{4})')
    for string in my_strings:
        if ((str(string)  != 'nan') and (float(string) <= 2018) and not(string in all_dates)):
            all_dates.append((string, index))
        index += 1 
        
    index = 0
    my_strings = df.str.extract(r'(\d{1,2} (Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec).?,? \d{2,4})')
    for string in my_strings[0]:
        if ((str(string)  != 'nan') and not(string in all_dates)):
            all_dates.append((string, index))
        index += 1 
        
    index = 0
    my_strings = df.str.extract(r'(\d{1,2} (January|February|March|April|May|June|July|August|September|October|November|December),? \d{2,4})')
    for string in my_strings[0]:
        if ((str(string)  != 'nan') and not(string in all_dates)):
            all_dates.append((string, index))
        index += 1 
        
    print(all_dates)
    print(len(all_dates))
    
    
    
    return # Your answer here

date_sorter()

Something is wrong


SystemExit: [datetime.date(93, 3, 25), datetime.date(85, 6, 18), datetime.date(71, 7, 8), datetime.date(75, 9, 27), datetime.date(96, 2, 6), datetime.date(79, 7, 6), datetime.date(78, 5, 18), datetime.date(89, 10, 24), datetime.date(86, 3, 7), datetime.date(71, 4, 10), datetime.date(85, 5, 11), datetime.date(75, 4, 9), datetime.date(98, 8, 1), datetime.date(72, 1, 26), datetime.date(1990, 5, 24), datetime.date(2011, 1, 25), datetime.date(82, 4, 12), datetime.date(1976, 10, 13), datetime.date(98, 4, 24), datetime.date(77, 5, 21), datetime.date(98, 7, 21), datetime.date(79, 10, 21), datetime.date(90, 3, 3), datetime.date(76, 2, 11), datetime.date(1984, 7, 25), datetime.date(82, 4, 13), datetime.date(89, 9, 22), datetime.date(76, 9, 2), datetime.date(71, 9, 12), datetime.date(86, 10, 24), datetime.date(1985, 3, 31), datetime.date(72, 7, 20), datetime.date(87, 4, 12), datetime.date(91, 6, 20), datetime.date(2012, 5, 12), datetime.date(83, 3, 15), datetime.date(73, 2, 14), datetime.date(88,

/Users/SteveXiong/anaconda3/envs/Coursera-Data-Intro/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2886: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
